In [61]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                run_prefix = "%# "\n                if line.startswith(run_prefix):\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def close(self):\n        self.inq_f.close()\n        os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

In [62]:
!rm ./a.exe

In [63]:
%%cpp main.c
%run gcc -fsanitize=address -DLOGP -std=c99 -Wall -Werror main.c -o a.exe
%run ./a.exe

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>


#if defined(LOGP)
    #define eprintf(...) fprintf(stderr, __VA_ARGS__)
#else
    #define eprintf(...) (void)42
#endif

#define logprintf_impl(fmt, line, ...) eprintf(__FILE__ ":" #line " " fmt, __VA_ARGS__)
#define logprintf_impl_2(line, fmt, ...) logprintf_impl(fmt "%s", line, __VA_ARGS__)
#define logprintf(...) logprintf_impl_2(__LINE__, __VA_ARGS__, "")


typedef int elem_t;

typedef struct {
    elem_t value;
    size_t prev;
    size_t next;
} node_t;

typedef struct {
    node_t* nodes;
    size_t max_index;
    size_t capacity;
    size_t free_top;
} arena_t;


void init_arena(arena_t* a) {
    a->nodes = NULL;
    a->max_index = 0;
    a->capacity = 0;
    a->free_top = -1;
}

void clear_arena(arena_t* a) {
    free(a->nodes);
    a->nodes = NULL;
    init_arena(a);
}

void reserve_arena(arena_t* a, size_t desired_size) {
    if (desired_size > a->capacity)
        a->capacity = desired_size;
    logprintf("resize to %d\n", (int)(a->capacity * sizeof(node_t)));
    a->nodes = realloc(a->nodes, a->capacity * sizeof(node_t));
}


size_t new_node_arena(arena_t* a) {
    size_t i;
    if (a->free_top != -1) {
        i = a->free_top;
        a->free_top = a->nodes[a->free_top].prev;
    } else {
        if (a->capacity == a->max_index) {
            reserve_arena(a, (a->capacity + 1) * 2);
        }
        i = a->max_index++;
    }
    return i;
}


void delete_node_arena(arena_t* a, size_t i) {
    a->nodes[i].prev = a->free_top;
    a->free_top = i;
}

typedef struct {
    arena_t* arena;
    size_t first;
    size_t last;
} list_t;

void init_list(list_t* d, arena_t* a) {
    d->arena = a;
    d->first = -1;
    d->last = -1;
}

elem_t pop_front_list(list_t* d);

void clear_list(list_t* d) {
    while (d->first != -1) {
        pop_front_list(d);
    }
}


void push_front_list(list_t* d, elem_t e) {  
    size_t i = new_node_arena(d->arena);
    d->arena->nodes[i].prev = -1;
    d->arena->nodes[i].next = d->first;
    d->arena->nodes[i].value = e;
    d->first = i;
    if (d->last == -1) {
        d->last = i;
    }
}

elem_t pop_front_list(list_t* d) {
    size_t i = d->first;
    if (i == d->last) {
        d->first = -1;
        d->last = -1;
    } else {
        d->first = d->arena->nodes[i].next;
        d->arena->nodes[d->first].prev = -1;
    }
    
    elem_t value = d->arena->nodes[i].value;
    delete_node_arena(d->arena, i);
    
    return value;
}

void print_list(list_t* d) {
    printf("[");
    for (int i = d->first; i != -1; i = d->arena->nodes[i].next) {
        printf("%d", d->arena->nodes[i].value);
        if (d->arena->nodes[i].next != -1) {
            printf(", ");
        }
    }
    printf("]\n");
}


int main() {
    arena_t a;
    
    init_arena(&a);
    reserve_arena(&a, 100);
    logprintf("x\n");
    const int N = 10;
    list_t d[N];
    for (int k = 0; k < N; ++k) {
        init_list(&d[k], &a);
        push_front_list(&d[k], 1);
        push_front_list(&d[k], 2); 
        push_front_list(&d[k], 3);
        for (int i = 0; i < 100; ++i) {
            push_front_list(&d[k], 42);
            if (i == 40) {
                for (int j = 0; j < 100; ++j) {
                    push_front_list(&d[k], 42);
                    assert(42 == pop_front_list(&d[k]));
                }
            }
            assert(42 == pop_front_list(&d[k]));
        }
        push_front_list(&d[k], 4);
    }
    
    for (int k = 0; k < N; ++k) {
        print_list(&d[k]); 
        clear_list(&d[k]);
    }

    clear_arena(&a);
    return 0;
}

Run: `gcc -fsanitize=address -DLOGP -std=c99 -Wall -Werror main.c -o a.exe`

Run: `./a.exe`

main.c:53 resize to 2400
main.c:143 x
[4, 3, 2, 1]
[4, 3, 2, 1]
[4, 3, 2, 1]
[4, 3, 2, 1]
[4, 3, 2, 1]
[4, 3, 2, 1]
[4, 3, 2, 1]
[4, 3, 2, 1]
[4, 3, 2, 1]
[4, 3, 2, 1]


In [64]:
%%cpp
typedef struct {
    int fd1;
    int fd2;
    void* ptr1;
    void* ptr2;
} state_t;

int work_not_finalized(state_t* s, args_t args) {
    ...
    s->fd1 = open(...);
    ...
        return -1;
    ...
            return -3;
    ...
    return 0;
}

int work(args_t args) {
    state_t state = {.fd1 = -1, .fd2 = -1, .ptr1 = NULL, .ptr2 = NULL};
    int ret_code = work_not_finalized(&state, args);
    close(state.fd1);
    close(state.fd2);
    free(state.ptr1);
    free(state.ptr2);
    return ret_code;
}




usage: ipykernel_launcher.py [-h] [--ejudge-style] fname
ipykernel_launcher.py: error: the following arguments are required: fname


SystemExit: 2

# Универсальное дерево отрезков

In [94]:
%%cpp main.c
%run gcc -fsanitize=address -DLOGP -std=c99 -Wall -Werror main.c -o a.exe
%run ./a.exe

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <stdint.h>


#if defined(LOGP)
    #define eprintf(...) fprintf(stderr, __VA_ARGS__)
#else
    #define eprintf(...) (void)42
#endif

#define logprintf_impl(fmt, line, ...) eprintf(__FILE__ ":" #line " " fmt, __VA_ARGS__)
#define logprintf_impl_2(line, fmt, ...) logprintf_impl(fmt "%s", line, __VA_ARGS__)
#define logprintf(...) logprintf_impl_2(__LINE__, __VA_ARGS__, "")


#define max(a, b) ((a) > (b) ? (a) : (b))

typedef int elem_t;
typedef elem_t (*aggregator_elem_t)(elem_t, elem_t);

typedef struct {
    elem_t* a;
    uint64_t lower_begin;
    aggregator_elem_t aggregator;
    elem_t neutral_element;
} segtree_t;

void segtree_init(segtree_t* t, int n, aggregator_elem_t aggr, elem_t nelem) {
    t->lower_begin = 1;
    while (t->lower_begin < n) {
        t->lower_begin *= 2;
    }
    t->a = calloc(t->lower_begin * 2, sizeof(elem_t));
    t->aggregator = aggr;
    t->neutral_element = nelem;
}

void segtree_destroy(segtree_t* t) {
    free(t->a);
}

void segtree_modify(segtree_t* t, uint64_t i, elem_t x) {
    i += t->lower_begin;
    t->a[i] = x;
    i /= 2;
    while (i > 0) {
        t->a[i] = t->aggregator(t->a[i * 2], t->a[i * 2 + 1]);
        i /= 2;
    }
}

elem_t segtree_get_impl(segtree_t* t, uint64_t i, uint64_t j, uint64_t v, uint64_t l, uint64_t r) {
    if (i <= l && r <= j) {
        return t->a[v];
    }
    if (r < i || j < l) {
        return t->neutral_element;
    }
    elem_t l_ans = segtree_get_impl(t, i, j, v * 2, l, (l + r) / 2);
    elem_t r_ans = segtree_get_impl(t, i, j, v * 2 + 1, (l + r) / 2 + 1, r);
    return t->aggregator(l_ans, r_ans);
}


elem_t segtree_get(segtree_t* t, uint64_t i, uint64_t j) {
    return segtree_get_impl(t, t->lower_begin + i, t->lower_begin + j, 
                            1, t->lower_begin, t->lower_begin * 2 - 1);
}

elem_t elem_max(elem_t a, elem_t b) {
    return (a > b) ? a : b;
}


elem_t elem_sum(elem_t a, elem_t b) {
    return a + b;
}

int main() {
    {
        segtree_t t;
        segtree_init(&t, 8, &elem_max, -1);
        segtree_modify(&t, 0, 0);
        segtree_modify(&t, 1, 2);
        segtree_modify(&t, 2, 1);
        segtree_modify(&t, 3, 4);
        segtree_modify(&t, 4, 3);
        segtree_modify(&t, 5, 7);
        segtree_modify(&t, 6, 5);
        segtree_modify(&t, 7, 4);
        assert(segtree_get(&t, 1, 5) == 7);
        assert(segtree_get(&t, 0, 2) == 2);
        assert(segtree_get(&t, 5, 7) == 7);
        assert(segtree_get(&t, 1, 3) == 4);
        assert(segtree_get(&t, 0, 7) == 7);
        assert(segtree_get(&t, 0, 0) == 0);

        segtree_destroy(&t);
    }
    
    {
        segtree_t t;
        segtree_init(&t, 8, &elem_sum, 0);
        segtree_modify(&t, 0, 1);
        segtree_modify(&t, 1, 2);
        segtree_modify(&t, 2, 3);
        segtree_modify(&t, 3, 4);
        assert(segtree_get(&t, 0, 3) == 10);
        assert(segtree_get(&t, 0, 0) == 1);
        assert(segtree_get(&t, 1, 2) == 5);
        assert(segtree_get(&t, 3, 3) == 4);

        segtree_destroy(&t);
    }

    return 0;
}

Run: `gcc -fsanitize=address -DLOGP -std=c99 -Wall -Werror main.c -o a.exe`

Run: `./a.exe`

# Дерево отрезков с присваиванием на отрезке

In [89]:
%%cpp main.c
%run gcc -fsanitize=address -DLOGP -std=c99 -Wall -Werror main.c -o a.exe
%run ./a.exe

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <stdint.h>
#include <stdbool.h>


#if defined(LOGP)
    #define eprintf(...) fprintf(stderr, __VA_ARGS__)
#else
    #define eprintf(...) (void)42
#endif

#define logprintf_impl(fmt, line, ...) eprintf(__FILE__ ":" #line " " fmt, __VA_ARGS__)
#define logprintf_impl_2(line, fmt, ...) logprintf_impl(fmt "%s", line, __VA_ARGS__)
#define logprintf(...) logprintf_impl_2(__LINE__, __VA_ARGS__, "")


#define max(a, b) ((a) > (b) ? (a) : (b))

typedef int elem_t;

typedef struct {
    elem_t value;
    bool do_propagate;
} segtree_node_t;

typedef struct {
    segtree_node_t* a;
    uint64_t lower_begin;
} segtree_t;

void segtree_init(segtree_t* t, int n) {
    t->lower_begin = 1;
    while (t->lower_begin < n) {
        t->lower_begin *= 2;
    }
    t->a = calloc(t->lower_begin * 2, sizeof(segtree_node_t));
}

void segtree_destroy(segtree_t* t) {
    free(t->a);
}

void segtree_push_down(segtree_t* t, uint64_t v) {
    if (t->a[v].do_propagate && v < t->lower_begin) {
        t->a[v * 2].value = t->a[v].value;
        t->a[v * 2 + 1].value = t->a[v].value;
        t->a[v * 2].do_propagate = true;
        t->a[v * 2 + 1].do_propagate = true;
    }
    t->a[v].do_propagate = false;
}

void segtree_modify_impl(segtree_t* t, uint64_t i, uint64_t j, elem_t x, uint64_t v, uint64_t l, uint64_t r) {
    if (r < i || j < l) {
        return;
    }
    if (i <= l && r <= j) {
        t->a[v].value = x;
        t->a[v].do_propagate = true;
        return;
    }

    segtree_push_down(t, v);
    segtree_modify_impl(t, i, j, x, v * 2, l, (l + r) / 2);
    segtree_modify_impl(t, i, j, x, v * 2 + 1, (l + r) / 2 + 1, r);
    t->a[v].value = max(t->a[v * 2].value, t->a[v * 2 + 1].value);
}

void segtree_modify_segment(segtree_t* t, uint64_t i, uint64_t j, elem_t x) {
    segtree_modify_impl(t, t->lower_begin + i, t->lower_begin + j, x,
                        1, t->lower_begin, t->lower_begin * 2 - 1);
    
}

void segtree_modify(segtree_t* t, uint64_t i, elem_t x) {
    segtree_modify_segment(t, i, i, x);
}

elem_t segtree_get_max_impl(segtree_t* t, uint64_t i, uint64_t j, uint64_t v, uint64_t l, uint64_t r) {
    if (r < i || j < l) {
        return -1;
    }
    segtree_push_down(t, v);
    if (i <= l && r <= j) {
        return t->a[v].value;
    }
    elem_t l_ans = segtree_get_max_impl(t, i, j, v * 2, l, (l + r) / 2);
    elem_t r_ans = segtree_get_max_impl(t, i, j, v * 2 + 1, (l + r) / 2 + 1, r);
    return max(l_ans, r_ans);
}


elem_t segtree_get_max(segtree_t* t, uint64_t i, uint64_t j) {
    return segtree_get_max_impl(t, t->lower_begin + i, t->lower_begin + j, 
                                1, t->lower_begin, t->lower_begin * 2 - 1);
}


int main() {
    segtree_t t;
    segtree_init(&t, 8);
    segtree_modify(&t, 0, 0);
    segtree_modify(&t, 1, 2);
    segtree_modify(&t, 2, 1);
    segtree_modify(&t, 3, 4);
    segtree_modify(&t, 4, 3);
    segtree_modify(&t, 5, 7);
    segtree_modify(&t, 6, 5);
    segtree_modify(&t, 7, 4);
    logprintf("segtree_get_max(&t, 1, 5) = %d\n", segtree_get_max(&t, 1, 5));
    assert(segtree_get_max(&t, 1, 5) == 7);
    assert(segtree_get_max(&t, 0, 2) == 2);
    assert(segtree_get_max(&t, 5, 7) == 7);
    assert(segtree_get_max(&t, 1, 3) == 4);
    assert(segtree_get_max(&t, 0, 7) == 7);
    assert(segtree_get_max(&t, 0, 0) == 0);
    
    segtree_modify_segment(&t, 1, 5, 8);
    assert(segtree_get_max(&t, 1, 5) == 8);
    assert(segtree_get_max(&t, 0, 2) == 8);
    assert(segtree_get_max(&t, 5, 7) == 8);
    assert(segtree_get_max(&t, 1, 3) == 8);
    assert(segtree_get_max(&t, 0, 7) == 8);
    assert(segtree_get_max(&t, 0, 0) == 0);
    assert(segtree_get_max(&t, 6, 6) == 5);
    assert(segtree_get_max(&t, 7, 7) == 4);
    
    
    segtree_destroy(&t);
    return 0;
}

Run: `gcc -fsanitize=address -DLOGP -std=c99 -Wall -Werror main.c -o a.exe`

Run: `./a.exe`

main.c:115 segtree_get_max(&t, 1, 5) = 7


In [88]:
!man calloc


MALLOC(3)                  Linux Programmer's Manual                 MALLOC(3)

NNAAMMEE
       malloc, free, calloc, realloc - allocate and free dynamic memory

SSYYNNOOPPSSIISS
       ##iinncclluuddee <<ssttddlliibb..hh>>

       vvooiidd **mmaalllloocc((ssiizzee__tt _s_i_z_e));;
       vvooiidd ffrreeee((vvooiidd _*_p_t_r));;
       vvooiidd **ccaalllloocc((ssiizzee__tt _n_m_e_m_b,, ssiizzee__tt _s_i_z_e));;
       vvooiidd **rreeaalllloocc((vvooiidd _*_p_t_r,, ssiizzee__tt _s_i_z_e));;
       vvooiidd **rreeaallllooccaarrrraayy((vvooiidd _*_p_t_r,, ssiizzee__tt _n_m_e_m_b,, ssiizzee__tt _s_i_z_e));;

   Feature Test Macro Requirements for glibc (see ffeeaattuurree__tteesstt__mmaaccrrooss(7)):

       rreeaallllooccaarrrraayy():
           Since glibc 2.29:
    